# Statistical Inference of Linear Regression Model

In [138]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
import pickle

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [139]:
def log_plus_one(x):
    return np.log(x+1)


In [140]:
df = pd.read_csv('../data/nyc_bldg2021_model_ready.csv')

with open("../pickle/lr_model.pickle", "rb") as f:
    lr = pickle.load(f)

with open("../pickle/X_train_ct.pickle", "rb") as f:
    X_train = pickle.load(f)

with open("../pickle/y_train.pickle", "rb") as f:
    y = pickle.load(f)

with open("../pickle/standard_scalar.pickle", "rb") as f:
    sc = pickle.load(f)


In [161]:
# calculating statistical-t and p_value of coefficients:
X0 = np.ones((X_train.shape[0],1))
X_train = X_train.drop(columns=[x for x in X_train.columns if 'energy_score' in x])
X = np.concatenate((X0, X_train), axis=1)
XTX_inv = np.linalg.inv(X.T @ X)
betas = XTX_inv @ (X.T @ y)
y_hat = X @ betas
resid_sq = (y - y_hat) ** 2
D = np.diag(resid_sq)
beta_vcov = XTX_inv @ (X.T @ D @ X) @ XTX_inv
np.diag(beta_vcov)
beta_se = np.sqrt(np.diag(beta_vcov))
t_stat = betas / beta_se
p_value = 2 * (1 - norm.cdf(np.abs(t_stat)))

In [162]:
features = ["Intercept"] + list(X_train.columns)

In [163]:
coef_stat = pd.DataFrame({
    "feature": features, 
    "coef": betas,
    "standard_error": beta_se, 
    "t_stat" : t_stat, 
    "p_value" : p_value

})

In [164]:
coef_stat.head()

,feature,coef,standard_error,t_stat,p_value
0,Intercept,76.148233,33.161657,2.296274,0.021660
1,property_gfa,7753.180307,6632.049127,1.169047,0.242385
2,multifamily_gfa,-6633.563635,6726.767440,-0.986144,0.324062
3,multifamily_cooled_area,-467.140292,710.010962,-0.657934,0.510581
4,building_age,2665.545797,686.951433,3.880254,0.000104


In [165]:

sc_df = pd.DataFrame({
    "feature": sc.get_feature_names_out(),
    "scaler": sc.scale_
})
coef_stat = pd.merge(coef_stat, sc_df, how="left", on="feature")
coef_stat.head()



,feature,coef,standard_error,t_stat,p_value,scaler
0,Intercept,76.148233,33.161657,2.296274,0.021660,NaN
1,property_gfa,7753.180307,6632.049127,1.169047,0.242385,196326.810960
2,multifamily_gfa,-6633.563635,6726.767440,-0.986144,0.324062,187493.752729
3,multifamily_cooled_area,-467.140292,710.010962,-0.657934,0.510581,36.850753
4,building_age,2665.545797,686.951433,3.880254,0.000104,32.492072


In [166]:
coef_stat['unscaled_coef'] =  coef_stat['coef']/coef_stat['scaler']
coef_stat.head(20)

,feature,coef,standard_error,t_stat,p_value,scaler,unscaled_coef
0,Intercept,76.148233,33.161657,2.296274,2.166025e-02,NaN,NaN
1,property_gfa,7753.180307,6632.049127,1.169047,2.423845e-01,1.963268e+05,3.949119e-02
2,multifamily_gfa,-6633.563635,6726.767440,-0.986144,3.240623e-01,1.874938e+05,-3.538018e-02
3,multifamily_cooled_area,-467.140292,710.010962,-0.657934,5.105806e-01,3.685075e+01,-1.267655e+01
4,building_age,2665.545797,686.951433,3.880254,1.043476e-04,3.249207e+01,8.203680e+01
5,multifamily_heated_area,-528.718417,350.729066,-1.507484,1.316867e-01,9.794054e+00,-5.398362e+01
6,Latitude,-541.896699,988.540859,-0.548178,5.835694e-01,8.138076e-02,-6.658782e+03
7,green_electricity_percent,2154.628550,752.901330,2.861767,4.212858e-03,5.302930e-01,4.063091e+03
8,Longitude,-2449.504953,970.556613,-2.523815,1.160891e-02,5.391860e-02,-4.542968e+04
9,property_gfa^2,47.963845,39.892825,1.202318,2.292405e-01,1.225515e+12,3.913771e-11


In [169]:
coef_stat.loc[coef_stat['p_value'] < 0.01, :]

,feature,coef,standard_error,t_stat,p_value,scaler,unscaled_coef
4,building_age,2665.545797,686.951433,3.880254,1.043476e-04,3.249207e+01,8.203680e+01
7,green_electricity_percent,2154.628550,752.901330,2.861767,4.212858e-03,5.302930e-01,4.063091e+03
12,property_gfa building_age,12.920898,4.240450,3.047058,2.310930e-03,1.296247e+07,9.967930e-07
16,property_gfa Longitude,13034.574157,4771.526239,2.731741,6.300063e-03,1.452198e+07,8.975753e-04
19,multifamily_gfa building_age,-29.173884,4.981281,-5.856704,4.721444e-09,1.261443e+07,-2.312740e-06
24,multifamily_cooled_area^2,7.295677,2.328372,3.133381,1.728052e-03,3.676098e+03,1.984625e-03
26,multifamily_cooled_area multifamily_heated_area,-10.211992,3.321998,-3.074051,2.111732e-03,3.688709e+03,-2.768446e-03
30,building_age^2,-14.692482,1.620671,-9.065681,0.000000e+00,4.039190e+03,-3.637482e-03
32,building_age Latitude,-792.174484,206.908555,-3.828621,1.288632e-04,1.324917e+03,-5.979048e-01
34,building_age Longitude,1857.329291,588.532841,3.155863,1.600237e-03,2.402710e+03,7.730144e-01


For every one percentage point increase "green_electricity_percent" feature, energy use will increase by 0.004 KBtu/Ft, which is an unexpected result...........
Older buildings consume more energy per area and it is expected,on average, when a building is older by one year its energy use per area is 0.8 KBtu.Sqft higher. This effect is nonlinear with deminishing marginal effects, as indicated by the negative sign of "building_age^2" feature.
Also from the interaction effects between "building_age" and geographic coordinates ("latitude" and "longtitude"), we can infere that the effect is location dependent. Such that, building_age has smaller effect as we go toward north and larger effect as we go far east........
Being located in Staten Island, there is a strong indicator of high energy use per area. This may be due to the fact that a lot of energy intesive buildings (such as power plants) are located in Staten Island.
Buildings categorized as subsidized housing have higher energy use on average by 6.32 KBtu/SqFt. One possible explanation for this observation could be that there is less investment on energy efficiency (for instance upgraded technology, insulation and so on) which are normally expensive and less affordable.